In [1]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from time import sleep

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set()

In [2]:
def generaPath(tipo, nombre):
    prepath = str(os.getcwd())
    #print(prepath)
    if (tipo == "I"):
        path = prepath + '\\' + nombre
    else :
        path = prepath + '\\generados\\' + nombre
    return path

In [3]:
df_cq_tr = pd.read_csv(generaPath('O', 'df_cq_tr.csv'))
df_cq_re = pd.read_csv(generaPath('O', 'df_cq_re.csv'))

df_cc_tr = pd.read_csv(generaPath('O', 'df_cc_tr.csv'))
df_cc_re = pd.read_csv(generaPath('O', 'df_cc_re.csv'))

df_ct_tr = pd.read_csv(generaPath('O', 'df_ct_tr.csv'))
df_ct_re = pd.read_csv(generaPath('O', 'df_ct_re.csv'))

In [4]:
df_cq_tr

,conteo,tupla,X,L,R,M,T,P,Pr,PrX,PrL,PrM
0,1,10|11|15|28|35,221,159,62,159|283,159|283,0.002,0.667,1.000,1.000,0.002
1,1,10|11|15|31|38,221,208,13,208|234,208|234,0.002,0.667,1.000,1.000,0.002
2,1,10|11|37|40|41,221,98,123,98|344,98|344,0.002,0.667,1.000,1.000,0.002
3,1,10|12|13|32|38,221,278,57,278|164,278|164,0.002,0.667,1.000,1.000,0.002
4,1,10|12|16|17|27,221,338,117,338|104,338|104,0.002,0.667,1.000,1.000,0.002
...,...,...,...,...,...,...,...,...,...,...,...,...
438,1,9|17|28|31|40,221,135,86,135|307,135|307,0.002,0.667,1.000,1.000,0.002
439,1,9|17|28|35|38,221,92,129,92|350,92|350,0.002,0.667,1.000,1.000,0.002
440,1,9|19|20|36|37,221,85,136,85|357,85|357,0.002,0.667,1.000,1.000,0.002
441,1,9|20|24|31|33,221,426,205,426|16,426|16,0.002,0.667,1.000,1.000,0.002


#### Introduzca datos para validar cuartetos y quintetos

In [5]:
def cayo_antestetos(conteos, df):
    media = []
    modaa = []
    lasts = []
    delta = []  
    resta = []
    for index, row in conteos.iterrows():
        sbs = str(row[1])
        splitedtupla= unsplited_tupla(sbs)
        sbs1 = [str(splitedtupla[0])]
        sbs2 = [str(splitedtupla[1])]
        sbs3 = [str(splitedtupla[2])]
        sbs4 = [str(splitedtupla[3])]
        sbs5 = [str(splitedtupla[4])]
        dftx = df[(df.n1.isin(sbs1) | df.n2.isin(sbs1) |df.n3.isin(sbs1) | df.n4.isin(sbs1) |df.n5.isin(sbs1)) 
                & (df.n1.isin(sbs2) | df.n2.isin(sbs2) |df.n3.isin(sbs2) | df.n4.isin(sbs2) |df.n5.isin(sbs2)) 
                & (df.n1.isin(sbs3) | df.n2.isin(sbs3) |df.n3.isin(sbs3) | df.n4.isin(sbs3) |df.n5.isin(sbs3))
                & (df.n1.isin(sbs4) | df.n2.isin(sbs4) |df.n3.isin(sbs4) | df.n4.isin(sbs4) |df.n5.isin(sbs4))
                & (df.n1.isin(sbs5) | df.n2.isin(sbs5) |df.n3.isin(sbs5) | df.n4.isin(sbs5) |df.n5.isin(sbs5)) ]
        #print(dftx)
        df_filt = dftx.iloc[0::1,[0]]
        df_tupla = calcular_deltas_verticales(df_filt)
        lasts.append(df_tupla[0])
        media.append(calc_media(df_tupla))
        resta.append(abs(int(df_tupla[0])- int(calc_media(df_tupla))))
        df_moda = calc_moda(df_tupla)
        pal = '|'.join([str(m) for m in df_moda]) if len(df_moda) > 1  else str(df_moda[0])
        modaa.append(pal)
        palabra = '|'.join([str(e) for e in df_tupla]) 
        delta.append(palabra)
    dft = conteos
    dft['X'] = media
    dft['L'] = lasts
    dft['R'] = resta
    dft['M'] = modaa
    df_filt2 = dft.iloc[0::1,[0]]
    #print("df_filt2", df_filt2)
    df_pesos = sumarcol_peso(df_filt2)
    #print("valor de df_pesos ", df_pesos)
    dft['T'] = delta
    dft['P'] = df_pesos
    proba, proba1,proba2,proba3 = how_hot_is(dft, df_tr_fu_filt_delta)
    dft['Pr']= proba
    dft['PrX'] = proba1
    dft['PrL'] = proba2
    dft['PrM'] = proba3
    return dft

In [6]:
def construir_quinteto(n1,n2,n3,n4,n5):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    return salida

In [7]:
def construir_cuarteto(n1,n2,n3,n4):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4)
    return salida

In [8]:
def construir_trio(n1,n2,n3):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3)
    return salida

In [9]:
def all_cuartetos(n1,n2,n3,n4,n5):
    cuarteto1 = construir_cuarteto(n1, n2, n3, n4)
    cuarteto2 = construir_cuarteto(n1, n2, n3, n5)
    cuarteto3 = construir_cuarteto(n1, n2, n4, n5)
    cuarteto4 = construir_cuarteto(n1, n3, n4, n5)
    cuarteto5 = construir_cuarteto(n2, n3, n4, n5)
    return cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5

In [10]:
def all_trios(n1,n2,n3,n4,n5):
    trio01 = construir_trio(n1,n2,n3)
    trio02 = construir_trio(n1,n2,n4)
    trio03 = construir_trio(n1,n2,n5)
    trio04 = construir_trio(n2,n3,n4)
    trio05 = construir_trio(n2,n3,n5)
    trio06 = construir_trio(n2,n4,n5)
    trio07 = construir_trio(n3,n4,n5)
    trio08 = construir_trio(n1,n3,n4)
    trio09 = construir_trio(n1,n3,n5)
    trio10 = construir_trio(n1,n4,n5)
    return trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10

In [11]:
def ingresar_sorteo_validar():
    n1 = int(input(f"ingresa el primer numero del sorteo: "))
    n2 = int(input(f"ingresa el segundo numero del sorteo: "))
    n3 = int(input(f"ingresa el tercer numero del sorteo: "))
    n4 = int(input(f"ingresa el cuarto numero del sorteo: "))    
    n5 = int(input(f"ingresa el quinto numero del sorteo: "))
    quinteto = construir_quinteto(n1, n2, n3, n4, n5)
    cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5 = all_cuartetos(n1, n2, n3, n4, n5)
    trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10 = all_trios(n1, n2, n3, n4, n5)
    return quinteto, cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5, trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10

In [12]:
def find_previous_coincidence(value_to_search, df):
    df_found = df.loc[df['tupla']== value_to_search]
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        return "Records Found: " + str(total_rows) +' - '+ str(value_to_search)
    else:
        return "No Records Found"

In [13]:
def validar_imprimir_resultados_sorteos(dictionary, propiedad, dfbase, df_to_search):
    dictionary[propiedad] = propiedad
    dictionary
    print(find_previous_coincidence(dfbase, df_to_search))

In [ ]:
i = 1
dictionary = {}
while i > 0:
    quinteto, cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5, trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10 = ingresar_sorteo_validar()
    validar_imprimir_resultados_sorteos(dictionary, "trios01tr", trio01, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios01re", trio01, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios02tr", trio02, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios02re", trio02, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios03tr", trio03, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios03re", trio03, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios04tr", trio04, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios04re", trio04, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios05tr", trio05, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios05re", trio05, df_ct_re)
    validar_imprimir_resultados_sorteos(dictionary, "trios06tr", trio06, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios06re", trio06, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios07tr", trio07, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios07re", trio07, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios08tr", trio08, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios08re", trio08, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios09tr", trio09, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios09re", trio09, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios10tr", trio10, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios10re", trio10, df_ct_re)
    print("Cuartetos")
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto1tr", cuarteto1, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto1re", cuarteto1, df_cc_re) 
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto2tr", cuarteto2, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto2re", cuarteto2, df_cc_re) 
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto3tr", cuarteto3, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto3re", cuarteto3, df_cc_re) 
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto4tr", cuarteto4, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto4re", cuarteto4, df_cc_re) 
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto5tr", cuarteto5, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto5re", cuarteto5, df_cc_re)
    print("Quintetos")
    validar_imprimir_resultados_sorteos(dictionary, "quintetotr", quinteto,df_cq_tr)
    validar_imprimir_resultados_sorteos(dictionary, "quintetore", quinteto,df_cq_re)
    
    i = int(input(f"ingresa 1 para continuar evaluando sorteos, 0 para parar: "))
    if i < 1:
        break
    


ingresa el primer numero del sorteo:  7
ingresa el segundo numero del sorteo:  16
ingresa el tercer numero del sorteo:  26
ingresa el cuarto numero del sorteo:  36
ingresa el quinto numero del sorteo:  29


No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 7|16|26
Records Found: 1 - 7|16|26
Records Found: 1 - 7|16|36
No Records Found
Records Found: 1 - 7|16|29
No Records Found
Records Found: 1 - 16|26|36
Records Found: 1 - 16|26|36
No Records Found
Records Found: 1 - 16|26|29
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found


ingresa 1 para continuar evaluando sorteos, 0 para parar:  1
ingresa el primer numero del sorteo:  1
ingresa el segundo numero del sorteo:  7
ingresa el tercer numero del sorteo:  16
ingresa el cuarto numero del sorteo:  29
ingresa el quinto numero del sorteo:  36


No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 1|7|16
Records Found: 1 - 1|7|16
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 7|16|29
No Records Found
Records Found: 1 - 7|16|36
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 1|16|29
No Records Found
No Records Found
No Records Found
No Records Found


ingresa 1 para continuar evaluando sorteos, 0 para parar:  1
ingresa el primer numero del sorteo:  2
ingresa el segundo numero del sorteo:  7
ingresa el tercer numero del sorteo:  23
ingresa el cuarto numero del sorteo:  28
ingresa el quinto numero del sorteo:  35


...

No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 1|7|16
Records Found: 1 - 1|7|16
No Records Found
No Records Found
Records Found: 1 - 1|7|35
No Records Found
Records Found: 1 - 7|16|29
No Records Found
Records Found: 1 - 7|16|35
Records Found: 1 - 7|16|35
Records Found: 1 - 7|29|35
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 1|16|29
No Records Found
No Records Found
Records Found: 1 - 1|29|35
No Records Found


ingresa 1 para continuar evaluando sorteos, 0 para parar:  1
ingresa el primer numero del sorteo:  1
ingresa el segundo numero del sorteo:  7
ingresa el tercer numero del sorteo:  16
ingresa el cuarto numero del sorteo:  28
ingresa el quinto numero del sorteo:  35


No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 1|7|16
Records Found: 1 - 1|7|16
No Records Found
Records Found: 1 - 1|7|28
Records Found: 1 - 1|7|35
No Records Found
No Records Found
No Records Found
Records Found: 1 - 7|16|35
Records Found: 1 - 7|16|35
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 1|28|35
No Records Found


ingresa 1 para continuar evaluando sorteos, 0 para parar:  1
ingresa el primer numero del sorteo:  2
ingresa el segundo numero del sorteo:  7
ingresa el tercer numero del sorteo:  16
ingresa el cuarto numero del sorteo:  28
ingresa el quinto numero del sorteo:  35


No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 2|7|16
Records Found: 1 - 2|7|28
Records Found: 1 - 2|7|28
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 7|16|35
Records Found: 1 - 7|16|35
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Records Found: 1 - 2|16|28
No Records Found
No Records Found
No Records Found
Records Found: 1 - 2|28|35


In [ ]:
print(find_previous_coincidence(quinteto, df_cq_tr))

In [ ]:
print(find_previous_coincidence(quinteto, df_cq_re))

In [ ]:
print(find_previous_coincidence(cuarteto1, df_cc_tr))

In [ ]:
print(find_previous_coincidence(cuarteto1, df_cc_re))

In [ ]:
print(find_previous_coincidence(cuarteto2, df_cc_tr))

In [ ]:
print(find_previous_coincidence(cuarteto2, df_cc_re))

In [ ]:
print(find_previous_coincidence(cuarteto3, df_cc_tr))

In [ ]:
print(find_previous_coincidence(cuarteto3, df_cc_re))

In [ ]:
print(find_previous_coincidence(cuarteto4, df_cc_tr))

In [ ]:
print(find_previous_coincidence(cuarteto4, df_cc_re))

In [ ]:
print(find_previous_coincidence(cuarteto5, df_cc_tr))

In [ ]:
print(find_previous_coincidence(cuarteto5, df_cc_re))